In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA

In [2]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)

In [3]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"
protien="CPSM_MOUSE"  
time=31
peptides=[x.split('_1_')[0] for x in os.listdir(path) if ((".csv" in x) and ("_1_" in x)) ]
peptides=[x  for x in peptides if 'A' in x]
all_AAs=np.unique([c for c in ("".join(sorted([x.replace('_','').replace('2','').replace('3','') for x in peptides])).lower())])
peptidesinfo=dict()
len(peptides)

24

In [4]:
def charcount(pepSeq):
    res={key: 0 for key in ['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y']}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [5]:
for k in range(len(peptides)):
    data=pd.read_csv(f"C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/{peptides[k]}_{time}_{protien}.csv")
    data["pxt_neh"]=data.NEH*data.pxt
    data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
    minval=min(data.RMSE)
    data=data[data.RMSE==minval]
    
    peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                           "charcount":charcount(peptides[k])}



In [18]:
all_AAs

array(['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y'], dtype='<U1')

In [21]:
visited=set()
visited.add(peptides[0].split('_')[1])
coefficients=[]
for k in range(1,len(peptides)):
    if(peptides[k].split('_')[1] in visited):
        continue
    pep1=peptidesinfo[peptides[0]]
    pep2=peptidesinfo[peptides[k]]
    visited.add(peptides[k].split('_')[1])
    r=pep1["pxt_neh"]/pep2["pxt_neh"]
    coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
    
    if(len(coefficients)==len(all_AAs)-1):
        break
print(len(coefficients))

19


In [22]:
if(len(coefficients)<len(all_AAs)-1):
    for k in range(len(peptides)-2,1,-1):
        pep1=peptidesinfo[peptides[len(peptides)-1]]
        pep2=peptidesinfo[peptides[k]]
        r=pep1["pxt_neh"]/pep2["pxt_neh"]
        coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
        if(len(coefficients)==len(all_AAs)-1):
            break

In [23]:
coefficients=np.array(coefficients)
coefficients[:,1:].shape

(19, 19)

In [27]:
a_val=3.7
coefficients=np.array(coefficients)
sol=list(np.linalg.solve(coefficients[:,1:],-a_val*coefficients[:,0])) 
sol.insert(0,a_val)
sol

[3.7,
 6.033722284230104,
 5.910602558242094,
 2.217639485684742,
 -1.1819257203533764,
 -0.12239919480750351,
 0.9428101948113912,
 -0.3589907728632624,
 4.44839045493928,
 0.8610045860954332,
 -0.28540514166519415,
 2.6885679237006226,
 1.414335195979446,
 5.609088125228098,
 2.921930447988651,
 -0.9794898659245731,
 -1.8738430715599579,
 0.47834593562109623,
 6.40485946372887,
 0.9712817321040138]

In [28]:
# pep1=peptidesinfo[peptides[7]]
# p_neh=0
# for index,c in enumerate(all_AAs):
#     p_neh+=pep1["charcount"][c]*sol[index]
# p_neh

# pep1=peptidesinfo[peptides[1]]
# p_neh=0
# for index,c in enumerate(all_AAs):
    
#     p_neh+=(pep1["charcount"][c]*sol[index])
# #     print(c,pep1["charcount"][c],sol[index],pep1["charcount"][c]*sol[index],"\t",p_neh)
# print(p_neh,pep1["T_NEH"])


for i in range (len(peptides)):
    pep1=peptidesinfo[peptides[i]]
    p_neh=0
    for index,c in enumerate(all_AAs):
#         print(c,pep1["charcount"][c],pep1["charcount"][c]*sol[index])
        p_neh+=(pep1["charcount"][c]*sol[index])
    print("\n",p_neh,"\t",pep1["T_NEH"])


 16.096866916176122 	 22.0

 18.371421162529604 	 25.0

 18.476411797876853 	 25.0

 23.305473972321927 	 34.0

 22.74556905836998 	 27.0

 21.415840947539078 	 27.0

 21.415840947539078 	 27.0

 23.147990381342346 	 33.0

 21.730779785002895 	 32.0

 13.717337991376489 	 19.0

 24.862656335515855 	 33.0

 17.14665954173507 	 21.0

 25.37005779587421 	 30.0

 19.683677341488124 	 24.0

 28.29198824386286 	 33.0

 21.870739479201912 	 30.0

 14.977091381612295 	 18.0

 19.24625074169768 	 25.0

 14.329717104603356 	 17.0

 20.471015336914565 	 25.0

 20.471015336914565 	 25.0

 18.861325495908574 	 23.0

 23.11823289360714 	 30.0

 12.312047450632832 	 28.0


In [11]:
[int(peptidesinfo[key]["T_NEH"]) for key in peptidesinfo.keys()]

[22,
 25,
 25,
 34,
 27,
 27,
 27,
 33,
 32,
 19,
 33,
 21,
 30,
 24,
 33,
 30,
 18,
 25,
 17,
 25,
 25,
 23,
 30,
 28]

In [41]:
pd.DataFrame(coefficients)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.247619,0.000000,1.000000,0.000000,-2.628572,0.123809,0.000000,1.123809,-0.876191,-0.876191,1.000000,-0.876191,0.123809,-0.876191,1.000000,-0.876191,1.000000,-0.752381,0.000000,1.000000
1,0.257577,0.000000,1.000000,-0.871212,-0.871212,1.000000,0.000000,1.128788,0.000000,-0.871212,0.128788,-0.871212,1.000000,-0.871212,0.128788,0.000000,0.128788,0.128788,0.000000,1.000000
2,-0.072071,0.000000,0.309310,0.000000,0.000000,-1.072071,0.000000,0.618619,0.000000,0.000000,1.000000,-1.381381,-0.381381,0.000000,0.309310,0.000000,-0.381381,1.000000,0.000000,1.000000
3,-0.123077,0.000000,1.000000,-1.415385,-0.707692,1.000000,0.000000,2.000000,0.000000,-1.415385,1.000000,-2.123077,1.000000,0.000000,1.000000,0.000000,0.292308,0.292308,0.000000,1.000000
4,1.248366,0.000000,1.000000,-1.503267,-1.503267,-0.503267,-0.751634,2.000000,-0.751634,-1.503267,1.000000,-0.751634,1.000000,-0.751634,1.000000,0.000000,1.000000,0.248366,0.000000,1.000000
5,1.248979,0.000000,1.000000,-1.502041,-1.502041,-0.502041,-0.751021,2.000000,-0.751021,-1.502041,1.000000,-0.751021,1.000000,-0.751021,1.000000,0.000000,1.000000,0.248979,0.000000,1.000000
6,1.304611,-0.695389,0.304611,-0.695389,-0.695389,0.304611,-0.695389,0.609221,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.304611,-2.781558,1.000000,-0.390779,-0.695389,1.000000
7,1.259260,-0.740740,1.000000,-0.740740,-0.740740,-0.481481,-1.481481,0.518519,0.000000,-1.481481,1.000000,-0.740740,0.259260,-0.740740,1.000000,0.000000,0.259260,0.259260,0.000000,1.000000
8,0.826531,0.000000,1.000000,-1.173469,0.000000,-0.173469,0.000000,-1.520406,-1.173469,-2.346937,1.000000,0.000000,-1.346937,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
9,0.057705,0.000000,0.352568,-1.294863,-0.647432,1.000000,0.000000,1.352568,-0.647432,-0.647432,0.352568,0.000000,0.352568,0.000000,1.000000,-1.294863,1.000000,0.352568,0.000000,1.000000


In [43]:
# (np.dot(coefficients[c],sol))/(np.dot(coefficients[c],sol))

In [44]:
for c in range( len(coefficients)):
    print(np.dot(coefficients[c],sol))

0.0
-8.881784197001252e-16
-8.881784197001252e-16
0.0
0.0
-1.7763568394002505e-15
1.1546319456101628e-14
-2.6645352591003757e-15
-5.329070518200751e-15
7.993605777301127e-15
8.881784197001252e-16
-1.2238310164022545e-15
-7.447976506632185e-15
1.3269108360302564e-15
-5.077436526492567e-15
3.552713678800501e-15
-1.2253963263291159e-15
8.881784197001252e-16
8.881784197001252e-16


In [93]:
pep1=peptidesinfo[peptides[0]]
pep2=peptidesinfo[peptides[1]]
r=pep1["pxt_neh"]/pep2["pxt_neh"]
r

0.6470588235294117

In [94]:
all_AAs

array(['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y'], dtype='<U1')